In [33]:
import pandas as pd
import numpy as np

In [65]:
en_df = pd.read_csv('/Users/kaitlinsinger/wa_notify/exposure_notif_timing.csv')

In [66]:
# make date time the index
en_df = en_df.set_index('Date Time')
en_df.drop(columns=['>1d'], inplace=True)
en_df.columns = en_df.columns.astype(int)
en_df['total_measures'] = en_df.sum(axis=1)

In [67]:
en_df

,1,5,15,30,60,120,180,360,720,1440,total_measures
Date Time,,,,,,,,,,,
11/30/20,0,0,0,0,0,0,0,0,0,0,0
12/1/20,0,0,0,0,0,0,0,0,0,0,0
12/2/20,0,0,0,0,0,0,0,0,0,0,0
12/3/20,0,0,0,0,0,0,0,0,0,0,0
12/4/20,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5/8/23,59,9,3,6,2,5,0,3,3,1,91
5/9/23,25,6,8,5,3,3,1,2,2,0,55
5/10/23,52,6,12,3,2,1,3,1,2,0,82


In [68]:
en_df_person_min = en_df.iloc[:, :10] * en_df.iloc[:, :10].columns
en_df_person_min['total_measures'] = en_df['total_measures']
en_df_person_min['total_minutes'] = en_df_person_min.sum(axis=1)
en_df_person_min['average_minutes'] = en_df_person_min.apply(
    lambda row: row['total_minutes'] / row['total_measures'] if row['total_measures'] > 0 else np.nan,
    axis=1
)

In [69]:
minute_values = en_df.iloc[:, :10].columns.astype(int)
median_minutes_per_day = []

for index, row in en_df.iloc[:, :10].iterrows():
    # Repeat each minute value according to the number of people in that category
    expanded_minutes = np.repeat(minute_values, row.values)
    
    # Calculate the median of the expanded list
    median_minutes_per_day.append(np.median(expanded_minutes))

en_df_person_min['median_minutes'] = median_minutes_per_day

/Users/kaitlinsinger/.pyenv/versions/3.10.0/envs/venv-inspace/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/kaitlinsinger/.pyenv/versions/3.10.0/envs/venv-inspace/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [70]:
en_df_person_min

,1,5,15,30,60,120,180,360,720,1440,total_measures,total_minutes,average_minutes,median_minutes
Date Time,,,,,,,,,,,,,,
11/30/20,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
12/1/20,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
12/2/20,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
12/3/20,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
12/4/20,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5/8/23,59,45,45,180,120,600,0,1080,2160,1440,91,5820,63.956044,1.0
5/9/23,25,30,120,150,180,360,180,720,1440,0,55,3260,59.272727,5.0
5/10/23,52,30,180,90,120,120,540,360,1440,0,82,3014,36.756098,1.0


In [71]:
rolling_avg_df = pd.DataFrame()

for day in [1,2,3,5,7,10,14,21,30]:
    colname = f"rolling_avg_avg_{day}_days"
    rolling_avg_df[colname] = en_df_person_min['average_minutes'].rolling(window=day).mean()
    colname = f"rolling_med_avg_{day}_days"
    rolling_avg_df[colname] = en_df_person_min['average_minutes'].rolling(window=day).median()
    colname = f"rolling_avg_med_{day}_days"
    rolling_avg_df[colname] = en_df_person_min['median_minutes'].rolling(window=day).mean()
    colname = f"rolling_med_med_{day}_days"
    rolling_avg_df[colname] = en_df_person_min['median_minutes'].rolling(window=day).median()

In [72]:
rolling_avg_df

,rolling_avg_avg_1_days,rolling_med_avg_1_days,rolling_avg_med_1_days,rolling_med_med_1_days,rolling_avg_avg_2_days,rolling_med_avg_2_days,rolling_avg_med_2_days,rolling_med_med_2_days,rolling_avg_avg_3_days,rolling_med_avg_3_days,...,rolling_avg_med_14_days,rolling_med_med_14_days,rolling_avg_avg_21_days,rolling_med_avg_21_days,rolling_avg_med_21_days,rolling_med_med_21_days,rolling_avg_avg_30_days,rolling_med_avg_30_days,rolling_avg_med_30_days,rolling_med_med_30_days
Date Time,,,,,,,,,,,,,,,,,,,,,
11/30/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12/1/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12/2/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12/3/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12/4/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5/8/23,63.956044,63.956044,1.0,1.0,79.941658,79.941658,1.0,1.0,83.977499,92.049180,...,1.285714,1.0,57.767391,55.446429,1.190476,1.0,57.750674,50.829768,1.133333,1.0
5/9/23,59.272727,59.272727,5.0,5.0,61.614386,61.614386,3.0,3.0,73.052015,63.956044,...,1.571429,1.0,58.291798,56.141304,1.380952,1.0,58.233432,52.577201,1.266667,1.0
5/10/23,36.756098,36.756098,1.0,1.0,48.014412,48.014412,3.0,3.0,53.328290,59.272727,...,1.285714,1.0,57.401783,56.141304,1.380952,1.0,57.799764,52.577201,1.266667,1.0


In [73]:
rolling_avg_df.to_csv('/Users/kaitlinsinger/wa_notify/rolling_EN_metrics.csv')

In [74]:
en_df_person_min.to_csv('/Users/kaitlinsinger/wa_notify/exposure_notif_timing_expanded.csv')